In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "llama2"

In [22]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
# chain.invoke("Tell me a joke")

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [7]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer would be:\n\n"My name is Santiago."'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("2024_GlobalMarketOutlook_NZ.pdf")
pages = loader.load_and_split()
pages


In [17]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("UK economy")

In [20]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [21]:
questions = [
    "What are the chances of recession in US?",
    "Is UK out of recession danger in 2024?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What are the chances of recession in US?
Answer: Based on the document provided, there is a possibility of recession in the US in 2024. According to the document, the US Federal Reserve's "higher for longer" approach will likely strain finances in the year ahead, impacting borrowers and refinancers. Additionally, the document states that elevated recession risks remain in 2024, despite markets pricing a more favourable soft-landing scenario.

However, it is important to note that the document does not provide a definitive answer to this question, and the chances of recession are subject to various factors such as economic growth, inflation, monetary policy, and geopolitical tensions. Therefore, the exact likelihood of recession in the US in 2024 cannot be determined with certainty based on the provided document alone.

Question: Is UK out of recession danger in 2024?
Answer: According to the document, the United Kingdom (UK) has the most challenging outlook of the major econo